# Joining NABD and NID

In [23]:
import geopandas as gp, pandas as pd, matplotlib as mpl
from pathlib import Path
gdrive = Path("/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/Data/bifurcation_data_repo") #where shapefiles/csv live 
# gdrive2 = Path("/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Jen_Dams/Matching") #where shapefiles/csv live 

### Read in data

In [24]:
# NABD
nabd_dams = gp.read_file(gdrive/"nabd/nabd_fish_barriers_2012.shp")  #read in NABD from Drive
nabd_dams = pd.DataFrame(nabd_dams)

#### filtering
nabd_dams = nabd_dams.drop_duplicates(subset='NIDID', keep="first")  #drop everything after first duplicate
nabd_dams = nabd_dams[['COMID', 'NIDID', 'Norm_stor', 'Max_stor','Year_compl', 'Purposes', 'geometry', 'Dam_name']]
nabd_dams.Norm_stor = nabd_dams.Norm_stor * 1233.48

# NID
nid = pd.read_csv(gdrive/'other_dam_datasets/NID2019_U.csv', usecols=['NIDID', 'NORMAL_STORAGE', 'MAX_STORAGE', 'YEAR_COMPLETED', 'DAM_NAME', 'LATITUDE', 'LONGITUDE'])

#### filtering
nid.NORMAL_STORAGE = nid.NORMAL_STORAGE * 1233.48   #convert units to cubic meters

### Figuring out what is missing

In [25]:
#Merge on NID
nid_by_ID = nid.merge(nabd_dams, on = 'NIDID', how = 'left')
nid_by_ID['COMID'] = nid_by_ID['COMID'].fillna(0)
missing = nid_by_ID[nid_by_ID['COMID'] ==0]

print('missing #',len(missing))

large_missing = missing[missing['NORMAL_STORAGE']>=10**8]  #is it 10^8 or 10^6?
print('large missing #', len(large_missing))
large_missing.to_csv('large_dams_missing.csv')

missing # 42648
large missing # 171


### Make NID look like NABD, so we can add in missing dams

In [26]:
nid_gdf = gp.GeoDataFrame(nid, geometry=gp.points_from_xy(nid.LONGITUDE, nid.LATITUDE))
nid_gdf
# 'COMID', 'NIDID', 'Norm_stor', 'Max_stor', 'Year_compl', 'Purposes', 'geometry'

,DAM_NAME,NIDID,LONGITUDE,LATITUDE,YEAR_COMPLETED,MAX_STORAGE,NORMAL_STORAGE,geometry
0,COOPER LAKE,AK00001,-149.823147,60.433708,1959.0,127600.0,1.381498e+08,POINT (-149.82315 60.43371)
1,BLUE LAKE,AK00002,-135.191700,57.063300,1961.0,266000.0,3.281057e+08,POINT (-135.19170 57.06330)
2,SALMON CREEK,AK00003,-134.403608,58.341850,1914.0,18000.0,1.480176e+07,POINT (-134.40361 58.34185)
3,ANNEX CREEK,AK00004,-134.126578,58.326939,1968.0,23400.0,2.886343e+07,POINT (-134.12658 58.32694)
4,CRYSTAL LAKE,AK00005,-132.845500,56.600000,1955.0,5800.0,6.414096e+06,POINT (-132.84550 56.60000)
...,...,...,...,...,...,...,...,...
91452,SHELL CREEK NO. 2,WY02601,-107.413988,44.516375,1957.0,1949.0,2.404053e+06,POINT (-107.41399 44.51637)
91453,WASHAKIE DIKE NO. 1,WY02902,-109.006036,42.979746,1935.0,10300.0,9.793831e+06,POINT (-109.00604 42.97975)
91454,WASHAKIE DIKE NO. 2,WY02903,-109.011705,42.979860,1935.0,10300.0,9.793831e+06,POINT (-109.01170 42.97986)
91455,WASHAKIE DIKE NO. 3,WY02904,-109.014574,42.979238,1935.0,10300.0,9.793831e+06,POINT (-109.01457 42.97924)


In [27]:
# nid_nhd_join = gp.sjoin(nid_gdf, flowlines, how="left", op='intersects')
# nid_nhd_join 

## Updating NIDID

In [28]:
# nabd_updateID = nabd_dams.copy()
# nabd_updateID.iloc[:,1]

In [29]:
# updated_ids = []
# nabd_ids = []
# wrong_id = pd.read_csv('/Users/rachelspinti/Documents/River_bifurcation/archive/Rachel_testing/large_dams_wrongID.csv', index_col = 0, usecols= [1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14,15])
# # wrong_id['NABD_NIDID'] = wrong_id['NABD_NIDID'].fillna(0)
# wrong_id = wrong_id[wrong_id['NABD_NIDID'].notna()]
# wrong_id
# for i in wrong_id.NABD_NIDID:
#     # if wrong_id.loc[i, 'NABD_NIDID'].notna():
#         # print(i)
#     nabd_ids.append(i)
# for i in wrong_id.NIDID:
#     updated_ids.append(i)
# # if nabd_updateID.NIDID == 'OK02500':
# #     nabd_updateID.NIDID = 'OK82901'
# # nabd_updateID
# nabd_ids
# updated_ids


In [30]:
# for i in nabd_updateID.NIDID:
#     if i in nabd_ids:
#         print(i)
#     #     nabd_updateID.loc[i,'NIDID'] = updated_ids[i]


## Junk

In [31]:
# wrong_id = pd.read_csv('/Users/rachelspinti/Documents/River_bifurcation/archive/Rachel_testing/large_dams_wrongID.csv', index_col = 0, usecols= [1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14,15])
# print(wrong_id.Dam_name)

In [32]:
# nid = pd.read_csv(gdrive/'other_dam_datasets/NID2019_U.csv')
# nid.columns
# nid.NORMAL_STORAGE

In [33]:
# nid.MAX_STORAGE